# Data preprocessing ([#2](https://github.com/molinari135/embryo-project/issues/2))

In this notebook, we perform the following operations:

1. Extract D202X folders from data/raw directory into data/processed
2. Remove empty extracted folders in data/processed
3. Fix folder name issue in data/processed
4. Rename extracted folders in data/processed

In order to perform all these operations it is **required** do to put in data/raw directory the folders that will be processed.

In [1]:
import re, shutil

from pathlib import Path
from loguru import logger
from embryo_project.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

2025-08-04 11:44:38.093 | INFO     | embryo_project.config:<module>:11 - PROJ_ROOT path is: C:\Users\Molinari\Desktop\embryo-project


In [2]:
# If needed, change folders path in embryo_project/config.py file
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# Automatically extract all D202X folders from DATASET_DIR
folder_pattern = re.compile(r'^D202[0-3]')
well_folder_pattern = re.compile(r'^D202[0-3].*WELL\d{1,2}')

found = 0
copied = 0
conflicts = 0

# Needed for those folders with long names
def to_long_path(path: Path) -> str:
    return r"\\?\{}".format(str(path.resolve()))

for folder in RAW_DATA_DIR.rglob("*"):
    if folder.is_dir() and well_folder_pattern.match(folder.name):
        found += 1
        destination = PROCESSED_DATA_DIR / folder.name

        if not destination.exists():
            shutil.copytree(to_long_path(folder), to_long_path(destination))
            copied += 1
        else:
            conflicts += 1

logger.info(f"Total number of folders founded: {found}")
logger.success(f"Copied folders: {copied}")
logger.warning(f"Conflicts (already copied): {conflicts}")

2025-08-04 11:44:54.540 | INFO     | __main__:<module>:24 - Total number of folders founded: 1166
2025-08-04 11:44:54.542 | SUCCESS  | __main__:<module>:25 - Copied folders: 1150
2025-08-04 11:44:54.542 | WARNING  | __main__:<module>:26 - Conflicts (already copied): 16


Every folder follows this pattern:

- `D[YEAR.MONTH.DAY]`
- `S[CODE]`
- `I[CODE]`
- `P` or `D`
- `WELL[CODE]`

Between every information, an underscore (`_`) or a whitespace (` `) can be found. Sometimes, folders have also an additional string `CAMPIONATO`, `CAMPIONATO_ok` `CAMPIONATO - Copia ([NUMBER])`.

Some examples:
- `DXXXX.XX.XX_SXXXXX_IXXXX_P_WELLXX_CAMPIONATO`
- `DXXXX.XX.XX_SXXXXX_IXXXX_P_WELLXX_CAMPIONATO_ok`
- `DXXXX.XX.XX_SXXXXX_IXXXX_P_WELLXX_CAMPIONATO - Copia (4)`
- `DXXXX.XX.XX_SXXXXX_IXXXX_P_WELLXX - CAMPIONATO`
- `DXXXX.XX.XX_SXXXXX_IXXXX_P_WELLXX`

In [4]:
# Delete empty folders
empty_deleted = 0

for folder in PROCESSED_DATA_DIR.iterdir():
    if folder.is_dir() and not any(folder.iterdir()):
        folder.rmdir()
        empty_deleted += 1
        logger.info(f"Deleted empty folder: {folder.name}")

logger.success(f"Deleted folders: {empty_deleted}")

2025-08-04 11:44:54.552 | INFO     | __main__:<module>:8 - Deleted empty folder: D2020.09.30_S00018_I4203_P_WELL01_CAMPIONATO - Copia (4)
2025-08-04 11:44:54.563 | INFO     | __main__:<module>:8 - Deleted empty folder: D2020.10.07_S00023_I4203_P_WELL16_CAMPIONATO
2025-08-04 11:44:54.626 | INFO     | __main__:<module>:8 - Deleted empty folder: D2021.10.20_S00083_I4203_P_WELL12
2025-08-04 11:44:54.626 | INFO     | __main__:<module>:8 - Deleted empty folder: D2021.10.20_S00083_I4203_P_WELL13
2025-08-04 11:44:54.627 | INFO     | __main__:<module>:8 - Deleted empty folder: D2021.10.20_S00083_I4203_P_WELL14
2025-08-04 11:44:54.627 | INFO     | __main__:<module>:8 - Deleted empty folder: D2021.10.20_S00083_I4203_P_WELL15
2025-08-04 11:44:54.631 | INFO     | __main__:<module>:8 - Deleted empty folder: D2021.11.10_S00084_I4203_P_WELL13
2025-08-04 11:44:54.632 | INFO     | __main__:<module>:8 - Deleted empty folder: D2021.11.10_S00084_I4203_P_WELL14
2025-08-04 11:44:54.632 | INFO     | __main__:

In [5]:
# Rename folders according to a common pattern
pattern = r'^(.*?WELL\d{1,2})'

pattern_matched = 0
renamed_folders = 0

for folderpath in PROCESSED_DATA_DIR.iterdir():
    if not folderpath.is_dir():
        continue

    relative_str = str(folderpath.relative_to(PROCESSED_DATA_DIR))
    match = re.match(pattern, relative_str)
    
    if match:
        cleaned_name = match.group(1)
        new_path = folderpath.parent / cleaned_name

        if folderpath.name != cleaned_name:
            # Rename only if names differ
            if not new_path.exists():
                folderpath.rename(new_path)
                renamed_folders += 1
            else:
                logger.warning(f"{new_path} already exists.")
        
        pattern_matched += 1
    else:
        logger.warning(f"No match for {relative_str}")

logger.info(f"Total folders: {len(list(PROCESSED_DATA_DIR.iterdir()))}")
logger.success(f"Matched: {pattern_matched}, Renamed: {renamed_folders}")

2025-08-04 11:44:54.948 | WARNING  | __main__:<module>:24 - C:\Users\Molinari\Desktop\embryo-project\data\processed\D2022.03.02_S00116_I4203_P_WELL01 already exists.
2025-08-04 11:44:55.157 | INFO     | __main__:<module>:30 - Total folders: 1104
2025-08-04 11:44:55.158 | SUCCESS  | __main__:<module>:31 - Matched: 1103, Renamed: 904


In [6]:
# Fix specific folder issue
old_to_new = {
    "D2022.03.02_S00116_I4203_P_WELL01": "D2022.03.02_S00116_I4203_P_WELL10",
    "D2022.03.02_S00116_I4203_P_WELL01_CAMPIONATO": "D2022.03.02_S00116_I4203_P_WELL01"
}

for old_name, new_name in old_to_new.items():
    old_path = PROCESSED_DATA_DIR / old_name
    new_path = PROCESSED_DATA_DIR / new_name

    if old_path.exists():
        if not new_path.exists():
            old_path.rename(new_path)
            logger.success(f"Renamed: {old_name} -> {new_name}")
        else:
            logger.warning(f"{new_name} already exists.")
    else:
        logger.warning(f"Folder not found: {old_name}")

2025-08-04 11:44:55.162 | SUCCESS  | __main__:<module>:14 - Renamed: D2022.03.02_S00116_I4203_P_WELL01 -> D2022.03.02_S00116_I4203_P_WELL10
2025-08-04 11:44:55.164 | SUCCESS  | __main__:<module>:14 - Renamed: D2022.03.02_S00116_I4203_P_WELL01_CAMPIONATO -> D2022.03.02_S00116_I4203_P_WELL01


In [7]:
original_folders = [p for p in RAW_DATA_DIR.rglob('*') if p.is_dir()]
processed_folders = [p for p in PROCESSED_DATA_DIR.iterdir() if p.is_dir()]

logger.info(f"Total number of original folders: {len(original_folders)}")
logger.info(f"Total number of processed folders: {len(processed_folders)}")
logger.info(f"{empty_deleted} out of {len(original_folders)-len(processed_folders)} folders were empty")

2025-08-04 11:44:55.607 | INFO     | __main__:<module>:4 - Total number of original folders: 1208
2025-08-04 11:44:55.608 | INFO     | __main__:<module>:5 - Total number of processed folders: 1103
2025-08-04 11:44:55.608 | INFO     | __main__:<module>:6 - 47 out of 105 folders were empty
